In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Grid

The `grid` module defines the top-level of `pytileproj`'s tiling system concept. A grid is a collection of tiling systems, which share a tiling concepts, but differ in their projection (however, the projection should be from the same family). An example is the [Equi7Grid](https://github.com/TUW-GEO/Equi7Grid) projection, which is a collection of seven continental equidistant azimuthal projections. 

## Regular grid

Within the `grid` module, the class `RegularGrid` contains a collection of `RegularProjTilingSystem` as on-the-fly generated class variables. Next to a `RegularProjTilingSystem`, the `RegularGrid` class was designed as the main entry point to handle projected tiling systems and to create them in a flexible and user-friendly way. 

### Initialisation

To create a `RegularGrid` object, one way is to define all the objects directly in Python. In principle, we can start with creating a simple `RegularGrid` object by using the same regular projected tiling system as defined in the [tiling system documentation](https://tuw-geo.github.io/pytileproj/guides/tiling_systemt.html):

In [ ]:
from pytileproj import (
    ProjSystemDefinition,
    RegularProjTilingSystem,
    RegularTilingDefinition,
)

name = "e7eu"
epsg = 27704
min_xy = (0, 0)
max_xy = None  # None will be filled with the projection bounds
axis_orientation = ("E", "S")
proj_def = ProjSystemDefinition(
    name=name, crs=epsg, min_xy=min_xy, max_xy=max_xy, axis_orientation=axis_orientation
)
tiling_defs = {
    1: RegularTilingDefinition(name="my_coarse_tiling", tile_shape=300_000),
    2: RegularTilingDefinition(name="my_fine_tiling", tile_shape=100_000),
}
rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=proj_def, tiling_defs=tiling_defs
)

With this, we are now able to initialise a `RegularGrid` object:

In [ ]:
from pytileproj import RegularGrid

rgrid = RegularGrid(**{name: rpts})

However, if we work with a grid system with several tiling systems beneath, then defining everything within Python may become tedious. Therefore, there are several classmethods, which allow to generate a `RegularGrid` object. 

`from_sampling` is exactly the same method as for a `RegularProjTilingSystem`, but now working with multiple `RegularTilingDefinition` instances. Here is an example:

In [ ]:
RegularGrid.from_sampling(
    {1: 1_000, 2: 10}, proj_defs={name: proj_def}, tiling_defs=tiling_defs
)

#### Export and import grid definitions

Another possibility is to start from a more generic regular grid definition. As an example, the core tiling system definitions of a `RegularGrid` object can be exported to a JSON file via

In [ ]:
from pathlib import Path

grid_def_path = Path("my_grid_def.json")
rgrid.to_grid_def(grid_def_path)

If we open it, it looks like this:

In [ ]:
import json
import pprint

with grid_def_path.open() as f:
    grid_def = json.load(f)

pprint.pprint(grid_def)  # noqa: T203

Now we can use this file to re-create the object by specifying the desired sampling. Like this it is convenient to share a grid definition with other people.

In [ ]:
RegularGrid.from_grid_def(grid_def_path, {1: 1_000, 2: 10})

If we want to be more explicit, there is also the option to export all class properties to a JSON file. This includes the sampling, thus allowing to exactly replicate the object, from which the JSON file was created.

In [ ]:
grid_path = Path("my_grid.json")
rgrid.to_file(grid_path)

Now the JSON file looks like this:

In [ ]:
with grid_path.open() as f:
    grid = json.load(f)

pprint.pprint(grid)  # noqa: T203

And the object can be re-created in the same manner:

In [ ]:
RegularGrid.from_file(grid_path)

#### Tiling system order

An optional argument available for all initalisation methods is `system_order`, which defines which tiling system should be used and how their order should be. This is important if users only want to work with a subset of the available tiling systems or if projection zones overlap and there should be a preference when querying for tiles and coordinates. In this example we add two tiling systems to a regular grid and define their order:

In [ ]:
af_def = ProjSystemDefinition(
    name="e7af",
    crs=27701,
    min_xy=min_xy,
    max_xy=max_xy,
    axis_orientation=axis_orientation,
)
eu_def = ProjSystemDefinition(
    name="e7eu",
    crs=27704,
    min_xy=min_xy,
    max_xy=max_xy,
    axis_orientation=axis_orientation,
)
af_rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=af_def, tiling_defs=tiling_defs
)
eu_rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=eu_def, tiling_defs=tiling_defs
)

rgrid = RegularGrid(
    system_order=["e7eu", "e7af"], **{rpts.name: rpts for rpts in [af_rpts, eu_rpts]}
)

### Supported samplings

If you are not sure, what samplings you can use for your grid design, you can fetch them directly via the `allowed_samplings` method with the desired tile size as an argument:

In [ ]:
RegularGrid.allowed_samplings(300)

### Tiling system access

The regular projected tiling system can then be accessed with:

In [ ]:
rgrid.e7eu  # ty: ignore[unresolved-attribute]

or equivalently,

In [ ]:
rgrid[name]

In case there are several tiling systems defining the regular grid and you want to know which tiling system(s) to use for a certain location, then you can use `get_systems_from_coord` or `get_systems_from_lonlat`. Here is an example with a coordinate inside the regular grid

In [ ]:
rgrid.get_systems_from_lonlat(16, 48)

and outside

In [ ]:
rgrid.get_systems_from_lonlat(-100, 48)

### Geographic coordinate transformation

A regular grid object also allows to transform geographic coordinates to grid coordinates. Usually, there is a unique result. But in case projection zones of different tiling systems overlap, coordinates are returned for each intersecting tiling system - with the order defined by `system_order`.

In [ ]:
rgrid.lonlat_to_xy(16, 48)

In [ ]:
grid_path.unlink(missing_ok=True)
grid_def_path.unlink(missing_ok=True)